In [1]:
from selenium import webdriver
import re
import pandas as pd
import time
from pygame import mixer

pygame 1.9.5
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#browser = webdriver.Firefox(executable_path='../geckodriver/geckodriver.exe')

In [2]:
def scrap_topic_content(nb_messages, topic_subject, browser):#20 messages par page
    messages_per_page = 20
    pages_to_scrap = nb_messages//messages_per_page
    messages = []
    for i in range(pages_to_scrap + 1):
        posts = browser.find_elements_by_class_name('txt-msg.text-enrichi-forum')
        for post in posts:
            try:
                initials = post.find_elements_by_tag_name('blockquote')
                actualised_post = post.text
                for initial in initials:
                    actualised_post = re.sub('{}'.format(initial.text), '', actualised_post)
                clean_post = actualised_post
            except:
                clean_post = post.text
            messages.append([clean_post, topic_subject])
        try:
            next_page_button = browser.find_element_by_class_name('xXx.pagi-suivant-actif')
            next_page_button.click()
        except:
            pass
    return messages

In [10]:
#TOPICS: fonction valable pour TOUT les catégories (18_25, 15_18 ,-15)
#changer l'url de départ
def open_topics(pages, browser = webdriver.Firefox(executable_path='../geckodriver/geckodriver.exe'), start_url = 'http://www.jeuxvideo.com/forums/0-51-0-1-0-01-0-blabla-18-25-ans.htm'):
    df = pd.DataFrame(columns=['Message', 'Topic_Subject'])
    browser.get(start_url)
    index_start_page = int(re.search(r'\d+(?=-0-blabla)', start_url)[0])
    for i in range(pages):
        for j in range(2,28):
            if j == 14:
                j += 1
            topic = browser.find_element_by_xpath('/html/body/div[2]/div[3]/div/div[2]/div[3]/div[1]/div[3]/ul/li[{}]/span[1]/a'.format(j))
            topic_subject = topic.text
            #print(topic.text)
            try:
                nb_messages = int(browser.find_element_by_xpath('/html/body/div[2]/div[3]/div/div[2]/div[3]/div[1]/div[3]/ul/li[{}]/span[2]'.format(j)).text)
                if nb_messages < 200:
                    #print("nb messages réels {}".format(nb_messages))
                    topic.click()
                    messages = scrap_topic_content(nb_messages, topic_subject, browser)
                    #print("Appending {} messages to dataframe".format(len(messages)))
                    df_len = len(df) + 1
                    for x in range(len(messages)):
                        df.loc[df_len + x] = messages[x]
            except:
                print("coundn't open topic")
                pass
            browser.get('http://www.jeuxvideo.com/forums/0-51-0-1-0-{}-0-blabla-18-25-ans.htm'.format(index_start_page + (i*25)))
        next_page_button = browser.find_element_by_xpath('/html/body/div[2]/div[3]/div/div[2]/div[3]/div[1]/div[3]/div[2]/div[2]/span/a')
        next_page_button.click()
    return df.drop_duplicates().reset_index(drop=True)

In [4]:
def actualise_and_save_df(scrapped_df):
    try:
        saved_df = pd.read_pickle('../Modo_bot/JVC.pkl')
        print("Dataframe opened successfully")
    except FileNotFoundError:
        return "Stored Database not found"
    to_save_df = pd.concat([saved_df, scrapped_df])
    to_save_df = to_save_df.drop_duplicates().reset_index(drop=True)
    to_save_df.to_pickle('../Modo_bot/JVC.pkl')
    return ("Dataframe stored successfully")

In [5]:
def check_stored_df():
    df = pd.read_pickle('../Modo_bot/JVC.pkl')
    return df

In [6]:
def jerome_rothen():
    mixer.init()
    mixer.music.load('C:/Users/grego/Downloads/enormebut.mp3')
    mixer.music.play()
jerome_rothen()

In [14]:
for i in range(0,5):
    df = open_topics(1)
    actualise_and_save_df(df)
jerome_rothen()
    #time.sleep(60)

coundn't open topic
Dataframe opened successfully
Dataframe opened successfully
Dataframe opened successfully
coundn't open topic
Dataframe opened successfully
Dataframe opened successfully


In [17]:
print(len(check_stored_df()))

4782


In [18]:
print(df['Message'])

0      "Un débutant, il va pas faire des serveurs en ...
1                                            Bah oui et?
2                   Il a raison. TypeScript master race.
3                              Etre prof en informatique
4      Je trouve que le PHP est un langage qui ira tr...
5      ?\ndonc tu confirmes que le php = noob\nles je...
6      \nsyntaxe immonde, code anarchique, rétrocompa...
7      Il a raison hein, PHP c'est pas très sérieux c...
8      Bah le plus simple c'est HTML\nEnsuite le php\...
9      PHP est beaucoup, beaucoup plus simple à mettr...
10     comme tout les ans j'ai été à Paris pour une r...
11           Les propos de l'auteur qui dérange le 18-25
12                                  \nc'est 100% no fake
13     T'es conscient que Paris est une ville tourist...
14     faut pas se balader dans les arrondissements d...
15                                        Respect à eux.
16     L’auteur qui va perdre son compte pour avoir d...
17                            J

In [9]:
df = check_stored_df()

In [16]:
df['Label'] = None

In [19]:
df

,Message,Topic_Subject
0,"""Un débutant, il va pas faire des serveurs en ...","Mon PROF: ""Le PHP c'est un langage de DEBUTANT"""
1,Bah oui et?,"Mon PROF: ""Le PHP c'est un langage de DEBUTANT"""
2,Il a raison. TypeScript master race.,"Mon PROF: ""Le PHP c'est un langage de DEBUTANT"""
3,Etre prof en informatique,"Mon PROF: ""Le PHP c'est un langage de DEBUTANT"""
4,Je trouve que le PHP est un langage qui ira tr...,"Mon PROF: ""Le PHP c'est un langage de DEBUTANT"""
5,?\ndonc tu confirmes que le php = noob\nles je...,"Mon PROF: ""Le PHP c'est un langage de DEBUTANT"""
6,"\nsyntaxe immonde, code anarchique, rétrocompa...","Mon PROF: ""Le PHP c'est un langage de DEBUTANT"""
7,"Il a raison hein, PHP c'est pas très sérieux c...","Mon PROF: ""Le PHP c'est un langage de DEBUTANT"""
8,Bah le plus simple c'est HTML\nEnsuite le php\...,"Mon PROF: ""Le PHP c'est un langage de DEBUTANT"""
9,"PHP est beaucoup, beaucoup plus simple à mettr...","Mon PROF: ""Le PHP c'est un langage de DEBUTANT"""


In [9]:
import sqlite3

In [19]:
db = sqlite3.connect('database.db')
cur = con.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS Posts (message TEXT, topic TEXT, label TEXT)')
db.commit()


Opened database successfully


In [25]:
oui = [tuple(x) for x in df.values]
oui

[('La France ne serait rien sans l’Afrique. Comme a dit Angélique Kidjo dans ONPC à Dupont Aignan, “Si vous aidiez vraiment l’Afrique, la France serait par terre” parce que la France organise l’exploitation de l’Afrique à son profit et ça c’est la vérité les petits bras. “Il faut dire la vérité aux Français” . Donc c’est à la France de payer pour accueillir les migrants.\n\n\n\n\nAh oui vraiment ?\nBien voyons donc la réalité:\nCommençons mes chers, si vous le voulez bien, par le commerce extérieur. Selon certains la France puiserait toutes ces richesses de l’Afrique, ne survit que grâce à l’Afrique, doit tout à l’Afrique etc etc on connait la chanson. .\n(Précisons ici que je ne compte que l’Afrique Subsaharienne et donc pas l’Afrique du Nord.)\nVoyons donc les chiffres\nLa France exporte pour 10 milliards de biens en Afrique et en importe pour 8 milliards. C’est à peine 2.4% des exportations françaises et 1.8% de ces importations. On est bien loin de l’image de la France qui ne survi

In [22]:
df[['Message', 'Topic_Subject', 'Label']]

,Message,Topic_Subject,Label
0,La France ne serait rien sans l’Afrique. Comme...,La France ne serait rien sans l'Afrique ! Nora...,None
1,Et hop une enorme pls pour eux,La France ne serait rien sans l'Afrique ! Nora...,None
2,je m'installe + pavé,La France ne serait rien sans l'Afrique ! Nora...,None
3,"post avant ""post avant""",La France ne serait rien sans l'Afrique ! Nora...,None
4,Et oui Kadhafi Sankara et Lumumba ont été suic...,La France ne serait rien sans l'Afrique ! Nora...,None
5,g pa lu lol,La France ne serait rien sans l'Afrique ! Nora...,None
6,\nRapport entre la France et Lumumba?\nEn plus...,La France ne serait rien sans l'Afrique ! Nora...,None
7,C'est un pavé que j'avais écris en juillet der...,La France ne serait rien sans l'Afrique ! Nora...,None
8,Le 12 avril 2019 à 16:02:12 _[Frans]_ a écrit ...,La France ne serait rien sans l'Afrique ! Nora...,None
9,la France n'a pas que colonisé l'Afrique hein,La France ne serait rien sans l'Afrique ! Nora...,None


In [ ]:
cur.execute('INSERT INTO Posts VALUES (?, ?, ?)', (message, topic, label))